## 추출된 관계 트리플 데이터 불러오기

In [1]:
import pandas as pd

df = pd.read_csv('../data/processed_data/relation_triples.csv', delimiter='\t')

In [2]:
df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation
0,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,모토로라 모빌리티,12,21,ORGANIZATION,P176
1,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,스마트폰,38,42,TERM,P279
2,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,미국,9,11,COUNTRY,P27
3,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,연극 배우,28,33,OCCUPATION,P106
4,"웨인 페더먼은 미국의 배우, 텔레비전 배우, 각본가, 성우, 영화배우, 영화 프로듀...",웨인 페더먼,0,6,PERSON,배우,12,14,OCCUPATION,P106


In [3]:
len(df)

750881

## 관계 종류별 개수 및 분포 비율 확인

In [4]:
rel_count_dict = dict(df.relation.value_counts())
list(rel_count_dict.items())[:5]

[('P17', 118924),
 ('P131', 79002),
 ('P530', 39216),
 ('P150', 34998),
 ('P47', 32643)]

In [5]:
len(rel_count_dict)

652

In [6]:
rel_proportion_dict = {}

for rel in rel_count_dict:
    rel_proportion_dict[rel] = rel_count_dict[rel] / len(df) * 100

In [7]:
list(rel_proportion_dict.items())[:5]

[('P17', 15.837929046013949),
 ('P131', 10.521241048847953),
 ('P530', 5.222665109384843),
 ('P150', 4.660924966805659),
 ('P47', 4.3472933793770245)]

In [8]:
list(rel_proportion_dict.items())[:100]

[('P17', 15.837929046013949),
 ('P131', 10.521241048847953),
 ('P530', 5.222665109384843),
 ('P150', 4.660924966805659),
 ('P47', 4.3472933793770245),
 ('P106', 4.280039047465578),
 ('P27', 4.077343813467114),
 ('P461', 2.6889746844040534),
 ('P279', 2.613596561905282),
 ('P495', 2.533557248085915),
 ('P641', 2.3879949019884643),
 ('P156', 2.295436960050927),
 ('P155', 2.2854486929353652),
 ('P527', 1.693876925904371),
 ('P361', 1.6089100669746603),
 ('P1376', 1.261984255827488),
 ('P36', 1.1770173968977775),
 ('P118', 0.8620540405204021),
 ('P1889', 0.8479372896637416),
 ('P31', 0.8459396362406293),
 ('P175', 0.659491983416813),
 ('P463', 0.5873101063950213),
 ('P54', 0.5857119836565314),
 ('P138', 0.5513523447789996),
 ('P81', 0.5343057022351079),
 ('P40', 0.5015441860960659),
 ('P159', 0.49208862656000085),
 ('P136', 0.47943682154695616),
 ('P171', 0.4410818758231997),
 ('P22', 0.42589970980754605),
 ('P26', 0.4149792044278654),
 ('P3373', 0.4132479047945014),
 ('P50', 0.40658906005

## wikidata의 relation 종류 가져오기 (쿼리 이용)

In [9]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?property ?propertyType ?propertyLabel ?propertyDescription ?propertyAltLabel WHERE {
  ?property wikibase:propertyType ?propertyType .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(xsd:integer(STRAFTER(STR(?property), 'P')))"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

results_en = results["results"]["bindings"]

results_en[0]

{'property': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P6'},
 'propertyType': {'type': 'uri',
  'value': 'http://wikiba.se/ontology#WikibaseItem'},
 'propertyLabel': {'xml:lang': 'en',
  'type': 'literal',
  'value': 'head of government'},
 'propertyDescription': {'xml:lang': 'en',
  'type': 'literal',
  'value': 'head of the executive power of this town, city, municipality, state, country, or other governmental body'},
 'propertyAltLabel': {'xml:lang': 'en',
  'type': 'literal',
  'value': 'president, chancellor, mayor, prime minister, governor, premier, first minister, executive power headed by, government headed by, head of national government'}}

In [10]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?property ?propertyType ?propertyLabel ?propertyDescription ?propertyAltLabel WHERE {
  ?property wikibase:propertyType ?propertyType .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],ko". }
}
ORDER BY ASC(xsd:integer(STRAFTER(STR(?property), 'P')))"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

results_ko = results["results"]["bindings"]

results_ko[0]

{'property': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P6'},
 'propertyType': {'type': 'uri',
  'value': 'http://wikiba.se/ontology#WikibaseItem'},
 'propertyLabel': {'xml:lang': 'ko', 'type': 'literal', 'value': '행정부 수반'},
 'propertyDescription': {'xml:lang': 'ko',
  'type': 'literal',
  'value': '항목 주제인 행정부 또는 지자체의 수장'},
 'propertyAltLabel': {'xml:lang': 'ko',
  'type': 'literal',
  'value': '군수, 구청장, 시장, 지사, 특별자치시장, 정부 대표, 정부 수반, 정부 수장, 행정부 수장, 광역시장, 도지사, 이곳의 행정부 수반, 지방 정부 대표, 지방 정부의 대표, 지방 정부의 장, 지자체 대표, 지자체 수장, 특별시장, 특별자치도지사, 현지사'}}

## relation 정보 추출

In [11]:
result_dict = {}

for i, rel_item in enumerate(results_ko):
    tmp_id = rel_item['property']['value'].split('http://www.wikidata.org/entity/')[1]
    
    tmp_dict = {}
    
    tmp_dict["relation_type"] = rel_item['propertyType']['value'].split('http://wikiba.se/ontology#')[1]

    tmp_dict["relation_label"] = rel_item['propertyLabel']['value']

    try:
        tmp_dict["relation_description"] = rel_item['propertyDescription']['value']
    except:
        pass
    
    result_dict[tmp_id] = tmp_dict


In [12]:
list(result_dict.items())[:5]

[('P6',
  {'relation_type': 'WikibaseItem',
   'relation_label': '행정부 수반',
   'relation_description': '항목 주제인 행정부 또는 지자체의 수장'}),
 ('P10',
  {'relation_type': 'CommonsMedia',
   'relation_label': '동영상 파일',
   'relation_description': '항목 주제와 관련된 동영상 (위키미디어 공용에 올라온 것에 한하며 유튜브 동영상인 경우 P1651를 이용하세요)'}),
 ('P14',
  {'relation_type': 'CommonsMedia',
   'relation_label': '노선 상징 모양',
   'relation_description': '항목 주제인 도로 노선을 상징하는 모양'}),
 ('P15',
  {'relation_type': 'CommonsMedia',
   'relation_label': '노선도',
   'relation_description': '항목 주제인 노선의 지도'}),
 ('P16',
  {'relation_type': 'WikibaseItem',
   'relation_label': '도로 노선의 등급',
   'relation_description': '항목 주제인 도로 노선이 속한 체계(등급)'})]

In [14]:
import json

with open('../data/relation/relid2label.json', 'w', encoding='utf-8') as f:
    json.dump(result_dict, f, indent=4)

In [15]:
result_list = []

for i, rel_item in enumerate(results_ko):
    tmp_dict = {}
    
    tmp_dict["relation_id"] = rel_item['property']['value'].split('http://www.wikidata.org/entity/')[1]
    
    tmp_dict["relation_type"] = rel_item['propertyType']['value'].split('http://wikiba.se/ontology#')[1]

    tmp_dict["relation_label"] = rel_item['propertyLabel']['value']

    try:
        tmp_dict["relation_description"] = rel_item['propertyDescription']['value']
    except:
        pass
    
    result_list.append(tmp_dict)

In [16]:
result_list[:5]

[{'relation_id': 'P6',
  'relation_type': 'WikibaseItem',
  'relation_label': '행정부 수반',
  'relation_description': '항목 주제인 행정부 또는 지자체의 수장'},
 {'relation_id': 'P10',
  'relation_type': 'CommonsMedia',
  'relation_label': '동영상 파일',
  'relation_description': '항목 주제와 관련된 동영상 (위키미디어 공용에 올라온 것에 한하며 유튜브 동영상인 경우 P1651를 이용하세요)'},
 {'relation_id': 'P14',
  'relation_type': 'CommonsMedia',
  'relation_label': '노선 상징 모양',
  'relation_description': '항목 주제인 도로 노선을 상징하는 모양'},
 {'relation_id': 'P15',
  'relation_type': 'CommonsMedia',
  'relation_label': '노선도',
  'relation_description': '항목 주제인 노선의 지도'},
 {'relation_id': 'P16',
  'relation_type': 'WikibaseItem',
  'relation_label': '도로 노선의 등급',
  'relation_description': '항목 주제인 도로 노선이 속한 체계(등급)'}]

In [17]:
rel_df = pd.DataFrame(result_list)
rel_df.head()

,relation_id,relation_type,relation_label,relation_description
0,P6,WikibaseItem,행정부 수반,항목 주제인 행정부 또는 지자체의 수장
1,P10,CommonsMedia,동영상 파일,항목 주제와 관련된 동영상 (위키미디어 공용에 올라온 것에 한하며 유튜브 동영상인 ...
2,P14,CommonsMedia,노선 상징 모양,항목 주제인 도로 노선을 상징하는 모양
3,P15,CommonsMedia,노선도,항목 주제인 노선의 지도
4,P16,WikibaseItem,도로 노선의 등급,항목 주제인 도로 노선이 속한 체계(등급)


In [18]:
rel_df.tail()

,relation_id,relation_type,relation_label,relation_description
9373,P10095,ExternalId,P10095,NaN
9374,P10096,ExternalId,P10096,NaN
9375,P10097,ExternalId,P10097,NaN
9376,P10098,ExternalId,P10098,NaN
9377,P10099,ExternalId,P10099,NaN


In [19]:
results_ko[-4]

{'property': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P10096'},
 'propertyType': {'type': 'uri',
  'value': 'http://wikiba.se/ontology#ExternalId'},
 'propertyLabel': {'type': 'literal', 'value': 'P10096'}}

## 기존 triple 데이터에 있는 relation id에 맞게 relation_label 연결

In [20]:
df.rename(columns={'relation': 'relation_id'}, inplace=True)
df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation_id
0,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,모토로라 모빌리티,12,21,ORGANIZATION,P176
1,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,스마트폰,38,42,TERM,P279
2,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,미국,9,11,COUNTRY,P27
3,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,연극 배우,28,33,OCCUPATION,P106
4,"웨인 페더먼은 미국의 배우, 텔레비전 배우, 각본가, 성우, 영화배우, 영화 프로듀...",웨인 페더먼,0,6,PERSON,배우,12,14,OCCUPATION,P106


In [21]:
import numpy as np

df['relation_label'] = df['relation_id'].apply(lambda x: result_dict[x]['relation_label'])
df['relation_type'] = df['relation_id'].apply(lambda x: result_dict[x]['relation_type'])
df['relation_description'] = df['relation_id'].apply(lambda x: result_dict[x]['relation_description'] if 'relation_description' in result_dict[x] else np.nan)


In [22]:
df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation_id,relation_label,relation_type,relation_description
0,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,모토로라 모빌리티,12,21,ORGANIZATION,P176,제조사,WikibaseItem,이 물건을 제조/제작한 주요 회사
1,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,스마트폰,38,42,TERM,P279,다음의 하위 개념임,WikibaseItem,항목 주제는 다음의 개념을 특징에 따라 다시 정의한 하위 개념임 (이 항목을 범주로...
2,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,미국,9,11,COUNTRY,P27,국적,WikibaseItem,대상 인물의 국적
3,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,연극 배우,28,33,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업
4,"웨인 페더먼은 미국의 배우, 텔레비전 배우, 각본가, 성우, 영화배우, 영화 프로듀...",웨인 페더먼,0,6,PERSON,배우,12,14,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업


In [100]:
for rel_id in rel_count_dict:
    print(f"{rel_id}: {rel_count_dict[rel_id]}개, label: {result_dict[rel_id]['relation_label']}")

P17: 118924개, label: 다음 나라의 것임
P131: 79002개, label: 다음 행정구역에 위치함
P530: 39216개, label: 수교국
P150: 34998개, label: 하위 행정구역
P47: 32643개, label: 다음과 경계를 맞닿음
P106: 32138개, label: 직업
P27: 30616개, label: 국적
P461: 20191개, label: 반대 개념
P279: 19625개, label: 다음의 하위 개념임
P495: 19024개, label: 처음 나온 나라
P641: 17931개, label: 관련 스포츠 종목
P156: 17236개, label: 다음
P155: 17161개, label: 이전
P527: 12719개, label: 다음으로 이루어져 있음
P361: 12081개, label: 다음의 한 부분임
P1376: 9476개, label: 다음 정부의 소재지임
P36: 8838개, label: 정부 소재지
P118: 6473개, label: 리그
P1889: 6367개, label: 다음과는 확실히 다름
P31: 6352개, label: 다음 종류에 속함
P175: 4952개, label: 연주자·가수
P463: 4410개, label: 다음 공동체의 구성원임
P54: 4398개, label: 소속 팀
P138: 4140개, label: 명칭의 유래
P81: 4012개, label: 이 정거장을 거쳐가는 노선
P40: 3766개, label: 자녀
P159: 3695개, label: 본부 위치
P136: 3600개, label: 장르
P171: 3312개, label: 다음의 하위 분류군임
P22: 3198개, label: 친아버지
P26: 3116개, label: 배우자
P3373: 3103개, label: 친형제자매
P50: 3053개, label: 저자
P30: 2970개, label: 대륙
P1532: 2966개, label: 대표국가
P178: 2960개, label: 개발 주체
P413: 2

비율이 0.1%를 넘는 relation에 대해서만 진행

In [23]:
final_rel_list = [item[0] for item in list(rel_proportion_dict.items())[:97]]
print(final_rel_list[:5])
print(len(final_rel_list))

['P17', 'P131', 'P530', 'P150', 'P47']
97


In [64]:
with open('../data/relation/relation_list.txt', 'w') as f:
    for rel in final_rel_list:
        f.write(rel + '\n')


In [66]:
with open('../data/relation/relation_list.txt', 'r') as f:
    lst = f.readlines()
lst = [x.strip() for x in lst]
lst

['P17',
 'P131',
 'P530',
 'P150',
 'P47',
 'P106',
 'P27',
 'P461',
 'P279',
 'P495',
 'P641',
 'P156',
 'P155',
 'P527',
 'P361',
 'P1376',
 'P36',
 'P118',
 'P1889',
 'P31',
 'P175',
 'P463',
 'P54',
 'P138',
 'P81',
 'P40',
 'P159',
 'P136',
 'P171',
 'P22',
 'P26',
 'P3373',
 'P50',
 'P30',
 'P1532',
 'P178',
 'P413',
 'P800',
 'P1365',
 'P276',
 'P1366',
 'P19',
 'P449',
 'P710',
 'P2936',
 'P1001',
 'P140',
 'P206',
 'P1056',
 'P20',
 'P6',
 'P123',
 'P1830',
 'P127',
 'P1659',
 'P112',
 'P101',
 'P3095',
 'P749',
 'P1696',
 'P137',
 'P2789',
 'P706',
 'P3842',
 'P39',
 'P425',
 'P1336',
 'P108',
 'P172',
 'P737',
 'P176',
 'P102',
 'P35',
 'P3730',
 'P1687',
 'P945',
 'P264',
 'P161',
 'P69',
 'P190',
 'P355',
 'P2341',
 'P664',
 'P407',
 'P793',
 'P840',
 'P1441',
 'P607',
 'P197',
 'P205',
 'P162',
 'P807',
 'P1269',
 'P170',
 'P97',
 'P750',
 'P551']

## 존재하는 relation 중 비율이 0.1% 이상인 97개만 선택

In [24]:
df['relation_id'] = df['relation_id'].apply(lambda x: x if x in final_rel_list else np.nan)
df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation_id,relation_label,relation_type,relation_description
0,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,모토로라 모빌리티,12,21,ORGANIZATION,P176,제조사,WikibaseItem,이 물건을 제조/제작한 주요 회사
1,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,스마트폰,38,42,TERM,P279,다음의 하위 개념임,WikibaseItem,항목 주제는 다음의 개념을 특징에 따라 다시 정의한 하위 개념임 (이 항목을 범주로...
2,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,미국,9,11,COUNTRY,P27,국적,WikibaseItem,대상 인물의 국적
3,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,연극 배우,28,33,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업
4,"웨인 페더먼은 미국의 배우, 텔레비전 배우, 각본가, 성우, 영화배우, 영화 프로듀...",웨인 페더먼,0,6,PERSON,배우,12,14,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750881 entries, 0 to 750880
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   sentence              750881 non-null  object
 1   subj_name             750881 non-null  object
 2   subj_start_pos        750881 non-null  int64 
 3   subj_end_pos          750881 non-null  int64 
 4   subj_type             750881 non-null  object
 5   obj_name              750881 non-null  object
 6   obj_start_pos         750881 non-null  int64 
 7   obj_end_pos           750881 non-null  int64 
 8   obj_type              750881 non-null  object
 9   relation_id           698327 non-null  object
 10  relation_label        750881 non-null  object
 11  relation_type         750881 non-null  object
 12  relation_description  733744 non-null  object
dtypes: int64(4), object(9)
memory usage: 74.5+ MB


In [26]:
df.dropna(subset=['relation_id'], inplace=True)
df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation_id,relation_label,relation_type,relation_description
0,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,모토로라 모빌리티,12,21,ORGANIZATION,P176,제조사,WikibaseItem,이 물건을 제조/제작한 주요 회사
1,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,스마트폰,38,42,TERM,P279,다음의 하위 개념임,WikibaseItem,항목 주제는 다음의 개념을 특징에 따라 다시 정의한 하위 개념임 (이 항목을 범주로...
2,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,미국,9,11,COUNTRY,P27,국적,WikibaseItem,대상 인물의 국적
3,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,연극 배우,28,33,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업
4,"웨인 페더먼은 미국의 배우, 텔레비전 배우, 각본가, 성우, 영화배우, 영화 프로듀...",웨인 페더먼,0,6,PERSON,배우,12,14,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업


In [27]:
df.reset_index(drop=True, inplace=True)
len(df)

698327

In [28]:
len(final_rel_list)

97

In [29]:
len(df), len(df[df['relation_id'].isna()])

(698327, 0)

## label에 따라 onehot label 만들기

In [30]:
def get_onehot(rel_id, rel_list=final_rel_list):
    onehot_list = [0] * len(rel_list)
    
    onehot_list[rel_list.index(rel_id)] = 1
    return onehot_list

In [31]:
final_rel_list[:5]

['P17', 'P131', 'P530', 'P150', 'P47']

In [32]:
get_onehot('P131')

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [33]:
df['label_onehot'] = df['relation_id'].apply(lambda x: get_onehot(x))
df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation_id,relation_label,relation_type,relation_description,label_onehot
0,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,모토로라 모빌리티,12,21,ORGANIZATION,P176,제조사,WikibaseItem,이 물건을 제조/제작한 주요 회사,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,스마트폰,38,42,TERM,P279,다음의 하위 개념임,WikibaseItem,항목 주제는 다음의 개념을 특징에 따라 다시 정의한 하위 개념임 (이 항목을 범주로...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,미국,9,11,COUNTRY,P27,국적,WikibaseItem,대상 인물의 국적,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,연극 배우,28,33,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"웨인 페더먼은 미국의 배우, 텔레비전 배우, 각본가, 성우, 영화배우, 영화 프로듀...",웨인 페더먼,0,6,PERSON,배우,12,14,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 데이터 split

In [34]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import numpy as np

def data_split(df, test_size=0.2):
    """ train/val/test dataset의 label의 분포 비율을 같게 split하는 함수.    
    """
    X = df.sentence.to_list()
    Y = df.label_onehot.to_list()
    
    msss = MultilabelStratifiedShuffleSplit(n_splits=2, test_size=test_size, random_state=0)
    
    for train_index, test_index in msss.split(X, Y):
        train_idx, test_idx = train_index, test_index
        
    test_df = df.iloc[test_idx.tolist()]
    tmp_df = df.iloc[train_idx.tolist()]
    
    X = tmp_df.sentence.to_list()
    Y = tmp_df.label_onehot.to_list()
    
    for train_index, test_index in msss.split(X, Y):
        train_idx, test_idx = train_index, test_index
    
    val_df = tmp_df.iloc[test_idx.tolist()]
    train_df = tmp_df.iloc[train_idx.tolist()]
    
    return train_df, val_df, test_df

In [35]:
train_df, val_df, test_df = data_split(df)

In [36]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 446926 entries, 1 to 698326
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   sentence              446926 non-null  object
 1   subj_name             446926 non-null  object
 2   subj_start_pos        446926 non-null  int64 
 3   subj_end_pos          446926 non-null  int64 
 4   subj_type             446926 non-null  object
 5   obj_name              446926 non-null  object
 6   obj_start_pos         446926 non-null  int64 
 7   obj_end_pos           446926 non-null  int64 
 8   obj_type              446926 non-null  object
 9   relation_id           446926 non-null  object
 10  relation_label        446926 non-null  object
 11  relation_type         446926 non-null  object
 12  relation_description  443061 non-null  object
 13  label_onehot          446926 non-null  object
dtypes: int64(4), object(10)
memory usage: 51.1+ MB


In [37]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111735 entries, 0 to 698323
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   sentence              111735 non-null  object
 1   subj_name             111735 non-null  object
 2   subj_start_pos        111735 non-null  int64 
 3   subj_end_pos          111735 non-null  int64 
 4   subj_type             111735 non-null  object
 5   obj_name              111735 non-null  object
 6   obj_start_pos         111735 non-null  int64 
 7   obj_end_pos           111735 non-null  int64 
 8   obj_type              111735 non-null  object
 9   relation_id           111735 non-null  object
 10  relation_label        111735 non-null  object
 11  relation_type         111735 non-null  object
 12  relation_description  110768 non-null  object
 13  label_onehot          111735 non-null  object
dtypes: int64(4), object(10)
memory usage: 12.8+ MB


In [38]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139666 entries, 6 to 698325
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   sentence              139666 non-null  object
 1   subj_name             139666 non-null  object
 2   subj_start_pos        139666 non-null  int64 
 3   subj_end_pos          139666 non-null  int64 
 4   subj_type             139666 non-null  object
 5   obj_name              139666 non-null  object
 6   obj_start_pos         139666 non-null  int64 
 7   obj_end_pos           139666 non-null  int64 
 8   obj_type              139666 non-null  object
 9   relation_id           139666 non-null  object
 10  relation_label        139666 non-null  object
 11  relation_type         139666 non-null  object
 12  relation_description  138459 non-null  object
 13  label_onehot          139666 non-null  object
dtypes: int64(4), object(10)
memory usage: 16.0+ MB


In [39]:
train_df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation_id,relation_label,relation_type,relation_description,label_onehot
1,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,스마트폰,38,42,TERM,P279,다음의 하위 개념임,WikibaseItem,항목 주제는 다음의 개념을 특징에 따라 다시 정의한 하위 개념임 (이 항목을 범주로...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,미국,9,11,COUNTRY,P27,국적,WikibaseItem,대상 인물의 국적,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"마저리 모나한은 미국의 텔레비전 배우, 영화배우, 연극 배우이다.",마저리 모나한,0,7,PERSON,연극 배우,28,33,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"2020년 1월 28일 당시 KIA 타이거즈 소속이었던 그, 키움 히어로즈 소속이었...",장영석,48,51,PERSON,키움 히어로즈,34,41,ORGANIZATION,P54,소속 팀,WikibaseItem,현재나 과거에 활동했던 스포츠 팀,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,서울 구로구 갑은 대한민국의 국회의원 선거구이다.,서울,0,2,CITY,대한민국,10,14,COUNTRY,P131,다음 행정구역에 위치함,WikibaseItem,"항목 주제가 위치한 행정 구역 (행정 구역이 아닌 값을 쓰고 싶거나, 사건이 일어난...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 라벨별 분포 확인

In [41]:
from collections import Counter

def get_rel_prop(df):
    counter = Counter(df['relation_id'])
    
    count_list = [(item[0], item[1], item[1] / len(df) * 100) for item in counter.items()]

    count_list.sort(key=lambda x: x[2], reverse=True)

    return count_list

In [44]:
get_rel_prop(df)[:5]

[('P17', 118924, 17.029844184744398),
 ('P131', 79002, 11.31303816120528),
 ('P530', 39216, 5.615707254624266),
 ('P150', 34998, 5.011692230144331),
 ('P47', 32643, 4.674457668112503)]

In [45]:
get_rel_prop(train_df)[:5]

[('P17', 76111, 17.029888616907495),
 ('P131', 50562, 11.313282288343036),
 ('P530', 25098, 5.615694768261412),
 ('P150', 22398, 5.0115679105713244),
 ('P47', 20891, 4.674375623705044)]

In [46]:
get_rel_prop(val_df)[:5]

[('P17', 19028, 17.029578914395668),
 ('P131', 12640, 11.312480422428067),
 ('P530', 6275, 5.615966348950642),
 ('P150', 5600, 5.011858414999777),
 ('P47', 5223, 4.674452946704256)]

In [47]:
get_rel_prop(test_df)[:5]

[('P17', 23785, 17.02991422393424),
 ('P131', 15800, 11.312703163260924),
 ('P530', 7843, 5.615539930978191),
 ('P150', 7000, 5.011957097647244),
 ('P47', 6529, 4.674723984362694)]

In [51]:
get_rel_prop(df)[-5:]

[('P1269', 848, 0.12143308220933745),
 ('P170', 839, 0.12014428770475721),
 ('P97', 831, 0.1189986925895748),
 ('P750', 821, 0.11756669869559676),
 ('P551', 768, 0.10997713105751317)]

In [48]:
get_rel_prop(train_df)[-5:]

[('P1269', 542, 0.1212728729140842),
 ('P170', 537, 0.12015411947391738),
 ('P97', 532, 0.11903536603375055),
 ('P750', 526, 0.11769286190555035),
 ('P551', 491, 0.10986158782438256)]

In [49]:
get_rel_prop(val_df)[-5:]

[('P1269', 136, 0.12171656150713742),
 ('P170', 134, 0.11992661207320893),
 ('P97', 133, 0.1190316373562447),
 ('P750', 131, 0.11724168792231618),
 ('P551', 123, 0.11008189018660224)]

In [50]:
get_rel_prop(test_df)[-5:]

[('P1269', 170, 0.1217189580857188),
 ('P170', 168, 0.12028697034353387),
 ('P97', 166, 0.11885498260134893),
 ('P750', 164, 0.117422994859164),
 ('P551', 154, 0.11026305614823936)]

## csv 파일로 저장

In [52]:
import os

os.mkdir('../data/data_split')

train_df.to_csv('../data/data_split/train.csv', index=False)
val_df.to_csv('../data/data_split/val.csv', index=False)
test_df.to_csv('../data/data_split/test.csv', index=False)

## toy dataset 만들기

In [53]:
val_df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type,relation_id,relation_label,relation_type,relation_description,label_onehot
0,모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.,모토로라 레이저 M,0,10,ARTIFACT,모토로라 모빌리티,12,21,ORGANIZATION,P176,제조사,WikibaseItem,이 물건을 제조/제작한 주요 회사,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"웨인 페더먼은 미국의 배우, 텔레비전 배우, 각본가, 성우, 영화배우, 영화 프로듀...",웨인 페더먼,0,6,PERSON,배우,12,14,OCCUPATION,P106,직업,WikibaseItem,항목 주제인 인물의 직업,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,"공부하는 곳인 명륜당이 앞에, 사당인 대성전이 뒤에 있는 전학후묘의 형태로 향교의 ...",뒤,26,27,TERM,앞,13,14,TERM,P461,반대 개념,WikibaseItem,이 항목과 반대 관계의 항목,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
9,서울 구로구 갑은 대한민국의 국회의원 선거구이다.,대한민국,10,14,COUNTRY,서울,0,2,CITY,P150,하위 행정구역,WikibaseItem,항목 주제인 행정 구역이 관할하는 하위 행정 구역,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13,"《2000 투데이》의 방송을 위해 영국의 BBC는 당시 총 5,000여명의 인력이 ...",텔레비전 센터,79,86,ARTIFACT,BBC,23,26,ORGANIZATION,P127,소유자,WikibaseItem,항목 주제를 소유하고 있는 사람/단체,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [54]:
len(val_df)

111735

In [56]:
toy_df = val_df

In [57]:
toy_train_df, toy_val_df, toy_test_df = data_split(toy_df)

In [58]:
len(toy_train_df), len(toy_val_df), len(toy_test_df)

(71503, 17881, 22351)

In [59]:
get_rel_prop(toy_train_df)[:5]

[('P17', 12178, 17.03145322573878),
 ('P131', 8090, 11.31421059256255),
 ('P530', 4016, 5.616547557445142),
 ('P150', 3584, 5.012377103058613),
 ('P47', 3342, 4.673929765184678)]

In [60]:
get_rel_prop(toy_val_df)[:5]

[('P17', 3044, 17.023656395056204),
 ('P131', 2022, 11.308092388568872),
 ('P530', 1004, 5.614898495609865),
 ('P150', 896, 5.010905430345059),
 ('P47', 836, 4.675353727420166)]

In [61]:
get_rel_prop(toy_test_df)[:5]

[('P17', 3806, 17.028320880497517),
 ('P131', 2528, 11.310455908013065),
 ('P530', 1255, 5.614961299270726),
 ('P150', 1120, 5.010961478233636),
 ('P47', 1045, 4.67540602210192)]

In [62]:
os.mkdir('../data/toy_data_split')

toy_train_df.to_csv('../data/toy_data_split/train.csv', index=False)
toy_val_df.to_csv('../data/toy_data_split/val.csv', index=False)
toy_test_df.to_csv('../data/toy_data_split/test.csv', index=False)